In [18]:
# https://towardsdatascience.com/creating-the-twitter-sentiment-analysis-program-in-python-with-naive-bayes-classification-672e5589a7ed
# Import Libraries
import os
import tweepy
from textblob import TextBlob
from wordcloud import WordCloud
import pandas as pd
import numpy as np
import csv
import time
import re
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import nltk
nltk.download('punkt') # https://www.nltk.org/data.html
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from string import punctuation 

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\shiya\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# Authentication Keys
# https://developer.twitter.com/en/portal/projects/1268606526274580480/apps/18064440/keys

# Think of these as the user name and password that represents
# your Twitter developer app when making API requests. 
consumerKey = '9gjPeGkMZj2935QAVEGEFqZG2'
consumerSecret = 'jSJbncxF8OifGepXcoR5uadfvhlVxXgoO9P5zDtMzNQidimRYx'

# User-specific credentials used to authenticate OAuth 1.0a API requests. 
# They specify the Twitter account the request is made on behalf of.
accessToken = '1130514658673213441-4GKVmzpYX67W2HlNiccXVIR3e0kzi7'
accessTokenSecret = 'HBN0T2QDg2nOcFOlZP0GyS9MwYGkjb3gtsrE7ebxKnJn8'

In [3]:
# Create the authentication object
auth = tweepy.OAuthHandler(consumerKey, consumerSecret) 
    
# Set the access token and access token secret
auth.set_access_token(accessToken, accessTokenSecret) 

# Creating the API object while passing in auth information
api = tweepy.API(auth)

In [4]:
# Constants 
START_DATE = '2020-05-01'
TWEET_NUMBER = 100
SEARCH_WORD = '#CRA'
RATE_LIMIT = 180
SLEEP_TIME = 900/180 # 15 minutes = 900 seconds

In [5]:
# Collect tweets using Cursor method
# http://docs.tweepy.org/en/v3.5.0/cursor_tutorial.html

def buildTestSet():
    tweet_list = []
    tweets_fetched = tweepy.Cursor(api.search,
                  q=SEARCH_WORD,
                  lang='en',
                  since=START_DATE).items(TWEET_NUMBER) 

    for tweet in tweets_fetched:
        tweet_list.append({"text":tweet.text, "label":None})
        print(tweet_list)
        
    # Array where Test Set is stored
    return tweet_list

In [19]:
# Build the test set
testDataSet = buildTestSet()

[{'text': '"I am no longer accepting the things I cannot change, I am changing the things I cannot accept."— Angela Davis \n✨ 🔥… https://t.co/EEeAwq7kMv', 'label': None}]
[{'text': '"I am no longer accepting the things I cannot change, I am changing the things I cannot accept."— Angela Davis \n✨ 🔥… https://t.co/EEeAwq7kMv', 'label': None}, {'text': 'RT @maxollieandme: wheat bag duck #ducks extra large lavender or no lavender wheat bag in cotton mustard fabric https://t.co/orEyVmbaBH via…', 'label': None}]
[{'text': '"I am no longer accepting the things I cannot change, I am changing the things I cannot accept."— Angela Davis \n✨ 🔥… https://t.co/EEeAwq7kMv', 'label': None}, {'text': 'RT @maxollieandme: wheat bag duck #ducks extra large lavender or no lavender wheat bag in cotton mustard fabric https://t.co/orEyVmbaBH via…', 'label': None}, {'text': 'RT @maxollieandme: Washable eco cotton face reusable wipes in modern floral design https://t.co/Izl217X71v via @Etsy #HandmadeHour #ListMyE

[{'text': '"I am no longer accepting the things I cannot change, I am changing the things I cannot accept."— Angela Davis \n✨ 🔥… https://t.co/EEeAwq7kMv', 'label': None}, {'text': 'RT @maxollieandme: wheat bag duck #ducks extra large lavender or no lavender wheat bag in cotton mustard fabric https://t.co/orEyVmbaBH via…', 'label': None}, {'text': 'RT @maxollieandme: Washable eco cotton face reusable wipes in modern floral design https://t.co/Izl217X71v via @Etsy #HandmadeHour #ListMyE…', 'label': None}, {'text': 'RT @maxollieandme: wheat bag duck #ducks extra large lavender or no lavender wheat bag in cotton mustard fabric https://t.co/orEyVmbaBH via…', 'label': None}, {'text': "RT @maxollieandme: Hold this and know I'm with you , isolation mindful lavender or chamomile sachet anxiety selfcare reminder mindfulness l…", 'label': None}, {'text': 'RT @maxollieandme: wheat bag duck #ducks extra large lavender or no lavender wheat bag in cotton mustard fabric https://t.co/orEyVmbaBH via…', 

[{'text': '"I am no longer accepting the things I cannot change, I am changing the things I cannot accept."— Angela Davis \n✨ 🔥… https://t.co/EEeAwq7kMv', 'label': None}, {'text': 'RT @maxollieandme: wheat bag duck #ducks extra large lavender or no lavender wheat bag in cotton mustard fabric https://t.co/orEyVmbaBH via…', 'label': None}, {'text': 'RT @maxollieandme: Washable eco cotton face reusable wipes in modern floral design https://t.co/Izl217X71v via @Etsy #HandmadeHour #ListMyE…', 'label': None}, {'text': 'RT @maxollieandme: wheat bag duck #ducks extra large lavender or no lavender wheat bag in cotton mustard fabric https://t.co/orEyVmbaBH via…', 'label': None}, {'text': "RT @maxollieandme: Hold this and know I'm with you , isolation mindful lavender or chamomile sachet anxiety selfcare reminder mindfulness l…", 'label': None}, {'text': 'RT @maxollieandme: wheat bag duck #ducks extra large lavender or no lavender wheat bag in cotton mustard fabric https://t.co/orEyVmbaBH via…', 

[{'text': '"I am no longer accepting the things I cannot change, I am changing the things I cannot accept."— Angela Davis \n✨ 🔥… https://t.co/EEeAwq7kMv', 'label': None}, {'text': 'RT @maxollieandme: wheat bag duck #ducks extra large lavender or no lavender wheat bag in cotton mustard fabric https://t.co/orEyVmbaBH via…', 'label': None}, {'text': 'RT @maxollieandme: Washable eco cotton face reusable wipes in modern floral design https://t.co/Izl217X71v via @Etsy #HandmadeHour #ListMyE…', 'label': None}, {'text': 'RT @maxollieandme: wheat bag duck #ducks extra large lavender or no lavender wheat bag in cotton mustard fabric https://t.co/orEyVmbaBH via…', 'label': None}, {'text': "RT @maxollieandme: Hold this and know I'm with you , isolation mindful lavender or chamomile sachet anxiety selfcare reminder mindfulness l…", 'label': None}, {'text': 'RT @maxollieandme: wheat bag duck #ducks extra large lavender or no lavender wheat bag in cotton mustard fabric https://t.co/orEyVmbaBH via…', 

[{'text': '"I am no longer accepting the things I cannot change, I am changing the things I cannot accept."— Angela Davis \n✨ 🔥… https://t.co/EEeAwq7kMv', 'label': None}, {'text': 'RT @maxollieandme: wheat bag duck #ducks extra large lavender or no lavender wheat bag in cotton mustard fabric https://t.co/orEyVmbaBH via…', 'label': None}, {'text': 'RT @maxollieandme: Washable eco cotton face reusable wipes in modern floral design https://t.co/Izl217X71v via @Etsy #HandmadeHour #ListMyE…', 'label': None}, {'text': 'RT @maxollieandme: wheat bag duck #ducks extra large lavender or no lavender wheat bag in cotton mustard fabric https://t.co/orEyVmbaBH via…', 'label': None}, {'text': "RT @maxollieandme: Hold this and know I'm with you , isolation mindful lavender or chamomile sachet anxiety selfcare reminder mindfulness l…", 'label': None}, {'text': 'RT @maxollieandme: wheat bag duck #ducks extra large lavender or no lavender wheat bag in cotton mustard fabric https://t.co/orEyVmbaBH via…', 

[{'text': '"I am no longer accepting the things I cannot change, I am changing the things I cannot accept."— Angela Davis \n✨ 🔥… https://t.co/EEeAwq7kMv', 'label': None}, {'text': 'RT @maxollieandme: wheat bag duck #ducks extra large lavender or no lavender wheat bag in cotton mustard fabric https://t.co/orEyVmbaBH via…', 'label': None}, {'text': 'RT @maxollieandme: Washable eco cotton face reusable wipes in modern floral design https://t.co/Izl217X71v via @Etsy #HandmadeHour #ListMyE…', 'label': None}, {'text': 'RT @maxollieandme: wheat bag duck #ducks extra large lavender or no lavender wheat bag in cotton mustard fabric https://t.co/orEyVmbaBH via…', 'label': None}, {'text': "RT @maxollieandme: Hold this and know I'm with you , isolation mindful lavender or chamomile sachet anxiety selfcare reminder mindfulness l…", 'label': None}, {'text': 'RT @maxollieandme: wheat bag duck #ducks extra large lavender or no lavender wheat bag in cotton mustard fabric https://t.co/orEyVmbaBH via…', 

[{'text': '"I am no longer accepting the things I cannot change, I am changing the things I cannot accept."— Angela Davis \n✨ 🔥… https://t.co/EEeAwq7kMv', 'label': None}, {'text': 'RT @maxollieandme: wheat bag duck #ducks extra large lavender or no lavender wheat bag in cotton mustard fabric https://t.co/orEyVmbaBH via…', 'label': None}, {'text': 'RT @maxollieandme: Washable eco cotton face reusable wipes in modern floral design https://t.co/Izl217X71v via @Etsy #HandmadeHour #ListMyE…', 'label': None}, {'text': 'RT @maxollieandme: wheat bag duck #ducks extra large lavender or no lavender wheat bag in cotton mustard fabric https://t.co/orEyVmbaBH via…', 'label': None}, {'text': "RT @maxollieandme: Hold this and know I'm with you , isolation mindful lavender or chamomile sachet anxiety selfcare reminder mindfulness l…", 'label': None}, {'text': 'RT @maxollieandme: wheat bag duck #ducks extra large lavender or no lavender wheat bag in cotton mustard fabric https://t.co/orEyVmbaBH via…', 

In [6]:
# data https://github.com/karanluthra/twitter-sentiment-training/blob/master/corpus.csv
# https://www.hugedomains.com/domain_profile.cfm?d=sananalytics&e=com
# https://github.com/shiyanboxer/twitter-sentiment-training
# For this task, we will be using the amazing Niek Sanders’ Corpus of over 5000 hand-classified tweets, which makes it quite reliable. There’s also a catch here. Twitter does not allow storing tweets on a personal device, even though all such data is publicly available. Therefore, the corpus includes a keyword (topic of the tweet), a label (pos/neg) and a tweet ID number for every tweet (i.e. row in our CSV corpus). You can get the file containing the corpus from the original site, or through this link of a personal repository.

def buildTrainingSet(corpusFile, tweetDataFile):
    corpus = []    
    with open(corpusFile,'rt') as csvfile:
        lineReader = csv.reader(csvfile,delimiter=',', quotechar="\"")
        for row in lineReader:
            corpus.append({"tweet_id":row[2], "label":row[1], "topic":row[0]})
    print(corpus[0])        
    trainingDataSet = []

    for tweet in corpus:
        try:
            status = api.get_status(tweet["tweet_id"])
            print("Tweet fetched" + status.text)
            tweet["text"] = status.text
            trainingDataSet.append(tweet)
            time.sleep(SLEEP_TIME) 
        except:
            continue
    # now we write them to the empty CSV file
    with open(tweetDataFile,'w') as csvfile:
        linewriter = csv.writer(csvfile,delimiter=',',quotechar="\"")
        for tweet in trainingDataSet:
            try:
                linewriter.writerow([tweet["tweet_id"], tweet["text"], tweet["label"], tweet["topic"]])
            except Exception as e:
                print(e)
    # print(trainingDataSet)
    return trainingDataSet

In [12]:
# corpusFile is the string path to the Niek Sanders’ CSV corpus file we downloaded. This file, as mentioned earlier, includes the tweet’s topic, label and id.
# tweetDataFile is the string path to the file we would like to save the full tweets in. In contrast to corpusFile, this file will include every tweet’s text as well as topic, label and id.

corpusFile = "C:\\Users\\shiya\\Documents\\Dev\\Twitter-Sentiment-Analysis\\twitter-sentiment-training-master\\twitter-sentiment-training-master\\corpus.csv"
tweetDataFile = "C:\\Users\\shiya\\Documents\\Dev\\Twitter-Sentiment-Analysis\\twitter-sentiment-training-master\\twitter-sentiment-training-master\\tweetDataFile.csv"

trainingData = buildTrainingSet(corpusFile, tweetDataFile)

{'tweet_id': '126415614616154112', 'label': 'positive', 'topic': 'apple'}
Tweet fetchedNow all @Apple has to do is get swype on the iphone and it will be crack. Iphone that is
Tweet fetchedHilarious @youtube video - guy does a duet with @apple 's Siri. Pretty much sums up the love affair! http://t.co/8ExbnQjY
Tweet fetched@RIM you made it too easy for me to switch to @Apple iPhone. See ya!
Tweet fetchedThe 16 strangest things Siri has said so far. I am SOOO glad that @Apple gave Siri a sense of humor! http://t.co/TWAeUDBp via @HappyPlace
Tweet fetchedGreat up close & personal event @Apple tonight in Regent St store!
Tweet fetchedFrom which companies do you experience the best customer service aside from @zappos and @apple?
Tweet fetchedLmao I think @apple is onto something magical! I am DYING!!! haha. Siri suggested where to find whores and where to hide a body lolol
Tweet fetchedRT @PhillipRowntree: Just registered as an @apple developer... Here's hoping I can actually do it... Any he

Tweet fetchedMeet #Siri, your new iPhone butler. Click the link and be amazed by all it can do: http://t.co/lvfFdCEL @Apple
Tweet fetchedSo, I am using my work PC (NEVER EVER) to get a feel for it; it has the worst speakers ever!! @apple you have spoiled me!! #imamac
Tweet fetchedI ♥ @Apple http://t.co/a8on3IAa
Tweet fetched@apple just got the new iOS5 upgrade with iMessage...good luck surviving now @BlackBerry
Tweet fetchedLoving #iOS5 !! #awesome @Apple
Tweet fetchedRT @MattyRiesz: @kathrynyee You were right, an iPhone is a must have. #addicted {WELCOME TO THE @APPLE CLUB}
Tweet fetched@blackberry boo hiss!............@apple wuhu!!!!!!!! When will my berry powered technology actually work??
Tweet fetched@Apple's Siri is witchcraft.  What's next @googleresearch. 2 yr lead lost?
Tweet fetched@Apple iOS 5 is sweet! Notifications, phone search covers mail now, wifi sync, iCloud backup and integrated Twitter are all well done.
Tweet fetchedRT @katebetts: Another great James Stewart story 

Tweet fetched@phxguy88 @Apple @BGR That's why all the ppl who stand in line for hrs to get the "newest" model are suckers...
Tweet fetchedWould it kill @apple to put a braille type bump on their earbuds so we know which bud is R and L in the dark.
Tweet fetched@APPLE Wow @MOTOROLA Just crushed your dreams....
Tweet fetchedOh, just fuck you, @apple. Already?? ---&gt;  iPhone 5 on schedule for summer launch? http://t.co/Ofh9PTaG via @BGR
Tweet fetched@apple, 
No, I won't wait until thursday for an available appointment just so a 'genius' can tell me I'm shit out of luck. #now
Tweet fetched@Apple, on the #iPad with #iOS5, why has the Messages Icon been included when it can't be used?
Tweet fetched@Steelo254 yea! I pre-order through @apple and they sorry too just like #AT&T
Tweet fetchedInteresting... @apple now requires you to have a reservation ?         #apple #iphone #4S    :  http://t.co/zZK4fTii
Tweet fetched@apple why is my iPhone battery so crappy #fail
Tweet fetchedMy @Apple @macb

Tweet fetchedI don't really like Siri's voice. Perhaps @apple can get Star Wars voices just like @garmin http://t.co/pSUJg9pN
Tweet fetched“@carlton858: I really hate dealing with the brain dead people at the @apple store. For such good products, customer service sucks.” in NZ?
Tweet fetchedI really hate dealing with the brain dead people at the @apple store. For such good products, customer service sucks.
Tweet fetchedAnyone else seeing missing signal bars on their #iPhone 3GS with an upgrade to #iOS5? cc/ @applecanada @apple @forstall
Tweet fetchedRT @CBM: Lies @apple. the battery on this new iPhone4S is definitely not any better.
Tweet fetchedUpdating to IOS5 with my regular iphone4 just made it slower and messed up.. @apple give me a free 4S and we can call it even, thanks
Tweet fetched@apple If you want to know what customers think dont send updates with the "noreply" return address. Who invented that anyway. #useless
Tweet fetchedEvery time I try the voice control on my iPod Touc

Tweet fetched@Apple can suck my dick gotta restore my ipad #annoyed
Tweet fetched@nansen Man, that sux! @apple io5 upgrade a total #fail for you!
Tweet fetchedDear @Apple Contacts, you are terrible. I just cleaned duplicates for 3 hours and they magically reappeared. Fix it.
Tweet fetchedPissed with whoever designs keyboards with @apple for not having a home and end key.  working on the CLI i use those keys often
Tweet fetched@NoVaTyler @apple Most of mine are gone too! Not recovered after I saved them b4 I upgraded my iPhone! Soooo pissed!
Tweet fetchedSiri is having a hard time remembering who my sister is. contact card needs space for family info. #iPhone4S @apple
Tweet fetched#ios5 getting sloppy @apple. U need to check your QA team. Too many issues with apps, data and syncing
Tweet fetchedRT @nansen: so this SMS full message is known @apple issue with new iOS5 update. trying restore. When all else #fail - restore.
Tweet fetchednot happy with @apple right now. the upgrade to my ito

...

Tweet fetchedRT @Jackiebaaby_: #'s are  #for #twitter #NOT #facebook. Duuuuhhh
Tweet fetched@_ahooten23 welcomee to #twitter
Tweet fetchedFinally off to bed..Nighty night #twitter world!
Tweet fetchedTwitter Buzz Builds for the Occupy Wall Street Movement [CHARTS] #twitter #occupywallstreet #teamfollowback #autofollow http://t.co/IRYn5Vis
Tweet fetchedRT @Gaabisfc_: My #twitter age is 219 days 12 hours 26 minutes 28 seconds. Find out yours at http://t.co/eOCFm1kD #twittertime
Tweet fetchedAccording to #twitter I have the same amount of followers as followees....#notgoingtolast
Tweet fetchedIs there a way that I can post voice notes on #twitter??
Tweet fetchedGN #Twitter Homies
Tweet fetchedI can't believe @mr_malie504 is not on #Twitter
Tweet fetched@95hm You can directly share your #Facebook photos on #Twitter with http://t.co/wYD5RnxN
Tweet fetched@StaceHendrix Pls let us know if you need help w/ music marketing promo. We got #Twitter & #Youtube on lock. http://t.co/EWFffl35
Tweet fe

Tweet fetchedRT @nagoul1: Can anyone please explain to us why the #Retweet "issue" is happening to many, but not to others? #twitter #tweeting #tweeps
Tweet fetched#twitter&facebookOff good night everyone   bye bye
Tweet fetchedCan anyone please explain to us why the #Retweet "issue" is happening to many, but not to others? #twitter #tweeting #tweeps
Tweet fetchedNiggas be in they feelings on #twitter hella funny
Tweet fetchedWowww | Can watching #Twitter trends help predict the future? http://t.co/2bRzlsGc via @zite
Tweet fetchedFacebook is becoming trashier and trashier #twitter
Tweet fetchedMy #twitter age is 2 years 0 days 3 hours 31 minutes 17 seconds. Find out yours at http://t.co/xNCmtWWb #twittertime
Tweet fetchedRT @SharonHayes: Top 50 #Twitter Acronyms, Abbreviations and Initialisms http://t.co/nEqHcJsY  /via @ruhanirabin
Tweet fetched«@BigBellyBigCuz RT @Free_Nuttball #fuckyoumean #Pittsburgh don't got the best #twitter.»
Tweet fetchedGoodNight #Twitter:)
Tweet fetchedwatchi

Tweet fetched#twitter/off
Tweet fetched#Twitter Thank you.Read those great tweets.helps live better! :) Bows.
Tweet fetchedI might call it a night goodnight #twitter
Tweet fetchedGood night to the #twitter world! #sleeptight
Tweet fetchedsearch #facebook for "FACEBOOK/TWITTER INDIE ARTIST MUSIC NETWORK (WORLDWIDE)
" add yourself for more #twitter followers! network here!
Tweet fetchedRT @iTweetTechNews: Famous wrestler Razor Ramon is trending on #Twitter.
Tweet fetchedGonna Crash Out Pretty Tired, Night #Twitter&lt;3
Tweet fetched#Twitter: What's happening? #Facebook: What's on your mind? #Myspace: Where is everyone?
Tweet fetched@FATBOYFRESH87 Hey brotha I forgot you had a #twitter
Tweet fetchedSo what #twitter needs is a "save tweet" text function. So I could "st @xyz 5" to save the 5th most recent msg on twitter site to read later
Tweet fetchedRT @SomalyMam: You follow me on #twitter, Like my #Foundation on #facebook http://t.co/3kGgPh7z! Together we can stop #humantrafficking
Tweet

Tweet fetchedWho will be our #Twitter #winner 4 #daytimewdonna giveaway for $100 this Fri. Tweet about our show, our guests, why you LOVE us use #dwd100
Tweet fetchedPut your twitter on autopilot and let it gain followers for you!! http://t.co/hze1TqAl #TeamFollowBack #Twitter #autopilot #1000aday 29
Tweet fetched#Oomf dnt start dis sh*t on #twitter
Tweet fetched@garbage Welcome to #Twitter!
Tweet fetchedGood night #Twitter and #TheLegionoftheFallen.  5:45am cimes awfully early!
Tweet fetchedTips for Reaching Out to Twitter's 50 Million Daily Users http://t.co/IF3IVLWN #twitter #mediagofer #socialmedia @SMsavvy @PRmate @KMiones
Tweet fetchedWhat do u think of Location Lock's new #twitter #logo? Can u see the cross hairs for when GPS tracking locks on 2 ur fleet vehicle?
Tweet fetchedGN #TWITTER
Tweet fetched#Twitter #Melbourne #Business "TWITTER 4 BUSINESS" November 16 http://t.co/wiZ4omU0 @KingstonCC @KeithKeller
Tweet fetchedTwitter Buzz Builds for the Occupy Wall Street Movement [CH

'charmap' codec can't encode characters in position 27-39: character maps to <undefined>
'charmap' codec can't encode characters in position 33-38: character maps to <undefined>
'charmap' codec can't encode characters in position 45-46: character maps to <undefined>
'charmap' codec can't encode characters in position 32-35: character maps to <undefined>
'charmap' codec can't encode characters in position 20-23: character maps to <undefined>
'charmap' codec can't encode characters in position 19-20: character maps to <undefined>
'charmap' codec can't encode characters in position 19-23: character maps to <undefined>
'charmap' codec can't encode characters in position 19-22: character maps to <undefined>
'charmap' codec can't encode characters in position 19-31: character maps to <undefined>
'charmap' codec can't encode character '\u267b' in position 19: character maps to <undefined>
'charmap' codec can't encode character '\u27a8' in position 86: character maps to <undefined>
'charmap' c

In [15]:
# pre-processing by first making all the text in lower-case letters

class PreProcessTweets:
    def __init__(self):
        self._stopwords = set(stopwords.words('english') + list(punctuation) + ['AT_USER','URL'])
        
    def processTweets(self, list_of_tweets):
        processedTweets=[]
        for tweet in list_of_tweets:
            processedTweets.append((self._processTweet(tweet["text"]),tweet["label"]))
        return processedTweets
    
    def _processTweet(self, tweet):
        tweet = tweet.lower() # convert text to lower-case
        tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', tweet) # remove URLs
        tweet = re.sub('@[^\s]+', 'AT_USER', tweet) # remove usernames
        tweet = re.sub(r'#([^\s]+)', r'\1', tweet) # remove the # in #hashtag
        tweet = word_tokenize(tweet) # remove repeated characters (helloooooooo into hello)
        return [word for word in tweet if word not in self._stopwords]

In [20]:
# First, we will create a variable that refers to it (an object), 
# and then call it on both the Training and Test sets  

tweetProcessor = PreProcessTweets()
preprocessedTrainingSet = tweetProcessor.processTweets(trainingData)
preprocessedTestSet = tweetProcessor.processTweets(testDataSet)

In [21]:
# This is just creating a list of all_words we have in the Training set, 
# breaking it into word features. 
# Those word_features are basically a list of distinct words, 
# each of which has its frequency (number of occurrences in the set) as a key.

def buildVocabulary(preprocessedTrainingData):
    all_words = []
    
    for (words, sentiment) in preprocessedTrainingData:
        all_words.extend(words)

    wordlist = nltk.FreqDist(all_words)
    word_features = wordlist.keys()
    
    return word_features

In [26]:
# Comparing every word against the tweet at hand, associating a number with the word with true or false

def extract_features(tweet):
    tweet_words=set(tweet)
    features={}
    for word in word_features:
        features['contains(%s)' % word]=(word in tweet_words)
    return features 

In [27]:
# Now we can extract the features and train the classifier
# This will build our final feature vector, with which we can proceed on to training.
# The NTLK built-in function apply_features does the actual feature extraction from our lists. 
# Our final feature vector is trainingFeatures.

word_features = buildVocabulary(preprocessedTrainingSet)
trainingFeatures=nltk.classify.apply_features(extract_features,preprocessedTrainingSet)

In [28]:
# Training the classifier
# Thanks to NLTK, it will only take us a function call to train the model as a Naive Bayes Classifier, 
# since the latter is built into the library:

NBayesClassifier=nltk.NaiveBayesClassifier.train(trainingFeatures)

In [29]:
NBResultLabels = [NBayesClassifier.classify(extract_features(tweet[0])) for tweet in preprocessedTestSet]

In [30]:
# get the majority vote

if NBResultLabels.count('positive') > NBResultLabels.count('negative'):
    print("Overall Positive Sentiment")
    print("Positive Sentiment Percentage = " + str(100*NBResultLabels.count('positive')/len(NBResultLabels)) + "%")
else: 
    print("Overall Negative Sentiment")
    print("Negative Sentiment Percentage = " + str(100*NBResultLabels.count('negative')/len(NBResultLabels)) + "%")

Overall Negative Sentiment
Negative Sentiment Percentage = 2.0%
